In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.svm import SVC
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.neural_network import MLPClassifier
from gensim.models import Word2Vec
from sklearn.linear_model import SGDClassifier
import pandas as pd
import numpy as np
import scipy.sparse as sp
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
import scipy.sparse as sp
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances, euclidean_distances
import pickle

In [10]:
#df_tweets = pd.read_csv("dframe.csv")
#df_ids = pd.read_csv("CMU_MisCov19_dataset.csv")
#df = pd.merge(df_tweets, df_ids, left_on='id', right_on='status_id')
#df = df[['text','id','status_id','annotation1','annotation2']]
#df.head()

,text,id,status_id,annotation1,annotation2
0,"News you can use, from a public health officia...",1236988271625936896,1236988271625936896,calling out or correction,NaN
1,#Twimbos are you aware of the fake COVID-19 he...,1236985619047419904,1236985619047419904,calling out or correction,calling out or correction
2,"No, lime juice won't immunize you: Some people...",1222615296344608768,1222615296344608768,calling out or correction,NaN
3,#Coronavirus Where to next? https://t.co/Xssp4...,1236981003224113152,1236981003224113152,news,ambiguous or hard to classify
4,Does cocaine help with coronavirus? WHO has th...,1236980398598557696,1236980398598557696,calling out or correction,NaN


In [4]:
def pd_load(inp):
    return pd.read_csv(inp)['text']

df = pd_load("https://raw.githubusercontent.com/CMU-IDS-2020/fp-ctqa/main/data/tweets.csv")

print(df.head())
origin = df

0                       TRUTH. https://t.co/Ou89dWvZxQ
1    May God grant you quick recovery. https://t.co...
2    @kavanagh_clare @simoncoveney @AddressingLife ...
3    this coronavirus shit has only gotten real in ...
4    Her thinking about how she should have stayed ...
Name: text, dtype: object


In [5]:

wnl = WordNetLemmatizer()
def lemmaSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(wnl.lemmatize(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)



In [6]:
pattern = re.compile(r"[A-Z][a-z]+|\d+|[A-Z]+(?![a-z])")

def split_hashtag(tag):
    return " ".join(pattern.findall(tag))

In [7]:
split_hashtag("@RealDonaldTrump")

'Real Donald Trump'

In [8]:

df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\'\|,;.!?=+-]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = ' '.join(split_hashtag(word) if word[0] is '@' or word[0] is '#' else word for word in text.split())
    ext = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE) #remove url
    text = lemmaSentence(text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwords from text
    return text.lower()

In [9]:
df = df.apply(clean_text)
df.head()

0                           truth http : co ou89dwvzxq
1    may god grant quick recovery http : co c19lxjyfvy
2    addressing life 19 fg ff never put vulnerable ...
3     coronavirus shit ha gotten real last day man wtf
4    her thinking stayed jail http : co vxz8glmgg6 ...
Name: text, dtype: object

In [16]:
def tfidf_vectorizer(reviews_train, reviews_test = None):
    tf_vectorizer = TfidfVectorizer(analyzer = str.split, tokenizer = str.split, preprocessor = lambda x : x)
    tf_train = tf_vectorizer.fit_transform(reviews_train)
    features = list(tf_vectorizer.vocabulary_.keys())
    features.sort()
    
    tf_test = None
    if reviews_test is not None:
        tf_test = tf_vectorizer.transform(reviews_test)
        
    return (tf_train, tf_test, features)

In [17]:
tf,test, features = tfidf_vectorizer(df)
n_topics = len(np.unique(df['annotation1']))
n_top_words = 3
lda = LatentDirichletAllocation(n_components=n_topics, learning_method='online', random_state=0)
lda.fit(tf)
top_words = []
features = np.array(features)
for topic in range(n_topics):
    sorted_indices = np.argsort(lda.components_[topic])
    top_words.append(list(features[sorted_indices[-n_top_words:]]))
    
top_words

[['anybody', 'oga', 'vaccination'],
 ['protesting', 'vahappened', 'antiva'],
 ['indeed', 'moking', '4hwcu'],
 ['hurch', 'affecting', '75cln'],
 ['stock', 'magic', 'uck'],
 ['storm', 'nited', 'riot'],
 ['seemingly', 'league', 'lobal'],
 ['coronavirus', 'http', 'tco'],
 ['wine', 'bleachhttps', 'book'],
 ['limit', 'google', 'stupidity'],
 ['il', 'ystem', 'mmune'],
 ['grain', 'ministry', 'black'],
 ['2', 'http', 'tco'],
 ['infect', 'ioeapon', 'working'],
 ['plus', 'ovt', 'shouldnt'],
 ['discussion', 'ocaine', 'ready']]

In [18]:
y = df['annotation1']
X = df['text']


skf = StratifiedKFold(n_splits=10, shuffle=True,random_state=0)


model = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-2, random_state=42, max_iter=15, tol=None)
for train_indices, test_indices in skf.split(X, y):
    X_train, X_test, features = tfidf_vectorizer(X[train_indices], X[test_indices])

    y_train = y[train_indices]
    y_test = y[test_indices]


    model.fit(X_train, y_train)
    print("Test Score {}".format(model.score(X_train,y_train)))
    print("Test Score {}".format(model.score(X_test,y_test)))

C:\Python36\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


Test Score 0.8695537364668603
Test Score 0.5581947743467933
Test Score 0.8711381040401374
Test Score 0.5391923990498813
Test Score 0.8753630842355427
Test Score 0.5581947743467933
Test Score 0.8714021653023502
Test Score 0.5463182897862233
Test Score 0.8698177977290732
Test Score 0.5653206650831354
Test Score 0.8745709004489042
Test Score 0.5558194774346793
Test Score 0.8721943490889886
Test Score 0.5249406175771971
Test Score 0.8782677581198838
Test Score 0.5771971496437055
Test Score 0.8685322069693769
Test Score 0.5547619047619048
Test Score 0.8748680042238648
Test Score 0.5595238095238095


In [10]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [11]:
X= df
tags = nlp(X[3])
print(X[3])

coronavirus shit ha gotten real last day man wtf


In [13]:
print([(X.text, X.ent_iob_, X.ent_type_) for X in tags])
print(type(tags[0]))
sentence_sets = [set([(X.text, X.ent_type_) for X in nlp(sentence)]) for sentence in df]
with open('ners.pkl', 'wb') as f:
    pickle.dump(sentence_sets, f)

[('coronavirus', 'O', ''), ('shit', 'O', ''), ('ha', 'O', ''), ('gotten', 'O', ''), ('real', 'O', ''), ('last', 'B', 'DATE'), ('day', 'I', 'DATE'), ('man', 'O', ''), ('wtf', 'O', '')]
<class 'spacy.tokens.token.Token'>


In [87]:
def jaccard_overlap(question_doc, sentence_doc):
    """
        Compute the jaccard overlap between the set of question tokens and the set of answer sentence tokens
        args:
            A (set): Set of words in the question sentence
            B (set): Set of words in the context_sentence
        return:
            jaccard_coefficient (float): The measure of jaccard overlap between the question and the context
    """
    return question_doc.similarity(sentence_doc)

def get_best_match_jaccard(question, df):
    """
        Given a row from the training dataframe, output the index of the sentence with highest
        jaccard overlap score
        args:
            df_row (pd.Dataframe): A row from the dataframe containing the training data.
        return:
            answer_sentence_index (int): The index of the sentence with the best jaccard overlap score
    """
    
    question = nlp(question)
    question_token_set = set([(X.text, X.ent_type_) for X in question])
    print(question_token_set)
    index = [jaccard_overlap(question, sentence_set) for sentence_set in sentence_sets]
    return np.argpartition(index, -5)[-5:]


g = "University of Texas".lower()
y = get_best_match_jaccard(g, X)

{('texas', 'ORG'), ('university', 'ORG'), ('of', 'ORG')}


C:\Python36\lib\site-packages\ipykernel_launcher.py:10: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  # Remove the CWD from sys.path while we load stuff.


In [88]:
origin[y]

827    Lydia Khalil discusses Trump’s response to COV...
135    @ieykinr @khairulaming Eh, no la. Indonesia la...
73     @the_hindu God bless your country!  @gauravbh ...
450    CBS News: Boris Johnson puts U.K. on coronavir...
83     @NRA @SteveGuest @RichardGrenell @kimguilfoyle...
Name: text, dtype: object

In [46]:

stop_words = set(stopwords.words('english'))
tfidf_vectorizer = TfidfVectorizer(tokenizer=nltk.word_tokenize,
                             stop_words=stop_words,
                             ngram_range=(1,2),
                             max_df=1.0,
                             min_df=10)
tfidf_vectorizer.fit_transform(X)

C:\Python36\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


<1177x448 sparse matrix of type '<class 'numpy.float64'>'
	with 18434 stored elements in Compressed Sparse Row format>

In [47]:
def get_tfidf_answer(question, df, vectorizer, distance_func):
    """
    Find the sentence in the context with the lowest distance from the question.
    
    args:
        datapoint (pandas.core.series.Series): A row from the training dataframe
        vectorizer (sklearn.feature_extraction.text.TfidfVectorizer): The tfidf_vectorizer model
        distance_func (function): similarity function that takes in the question embedding and sentence embeddings and
                                    returns a distance measurement between the two embeddings 
        
    return:
        int: The index of the context sentence with the lowest distance from the question
    """
    question_vector = vectorizer.transform([question])
    sentence_vectors = vectorizer.transform(df)
    index = [distance_func(question_vector,v) for v in sentence_vectors]
    return np.argmin(index)

In [48]:
get_tfidf_answer("Donald Trump",X,tfidf_vectorizer,cosine_distances)

943

In [49]:
X[943]

'donald trump lying murdering po http : co wcv5hhxcwq'